# Common_Anal_Merge

This notebook merges the combined distance data from the individual “fires_distance_{batch_no}.csv files joined with selected fire attribute data from USGS_Wildland_Fire_Combined_Dataset.csv.  After dropping data related to (1) fires prior to 1963; (2) fires further than 1250 miles from Kearney, Nebraska, and (3) fires that had a “curvedRIng” geometry, there were a total of 100972 fires.  Results saved to the file “fire_dat_merged.csv." 



# License
The code in this notebook was developed by Sue Boyd in response to Assignment 1 in DATA 512, a course in the UW MS Data Science degree program. The code in this notebook is provided under the MIT license located in the same repository as this notebook.




In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import geojson
import matplotlib.pyplot as plt
import time, json, folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
from pyproj import Transformer, Geod
from shapely.geometry import Polygon
from shapely.geometry import Point
from geopy.distance import geodesic
import re

# Step 1 Load and Clean Attributes Data  

In [2]:
# Load dataframe of attributes 

f = "Data/Wildfire_Attributes_Raw/USGS_Wildland_Fire_Combined_Dataset.csv"
columns_to_include = ["OID_", "Listed_Fire_Names", "Assigned_Fire_Type", "Fire_Year", "Listed_Fire_Dates", "GIS_Acres"]  # The 
attributes= pd.read_csv(f, usecols=columns_to_include)
attributes.head()


C:\Users\suetb\AppData\Local\Temp\ipykernel_8756\1271887303.py:5: DtypeWarning: Columns (0,2,3,10,14) have mixed types. Specify dtype option on import or set low_memory=False.
  attributes= pd.read_csv(f, usecols=columns_to_include)


,OID_,Assigned_Fire_Type,Fire_Year,GIS_Acres,Listed_Fire_Names,Listed_Fire_Dates
0,1,Wildfire,1860,3940.207089,Big Quilcene River (1),Listed Other Fire Date(s): 2006-11-02 - NIFC D...
1,2,Wildfire,1860,772.518249,Harrison Lake (1),Listed Other Fire Date(s): 2006-11-02 - NIFC D...
2,3,Wildfire,1860,333.020409,Tunnel Creek (1),Listed Other Fire Date(s): 2006-11-02 - NIFC D...
3,4,Wildfire,1870,22294.993577,1870 (2),Listed Other Fire Date(s): 2017-01-30 - NIFC D...
4,5,Wildfire,1870,36.985574,No Fire Name Provided (2),NaN


In [3]:
# inspect the shape of the datast 
attributes.shape

(444179, 6)

A few quirky entries in the file create duplicate rows
Clean that up by dropping duplicates 

In [4]:
attributes = attributes.drop_duplicates()
attributes.shape

(135072, 6)

We still have a few funky entries related to footnotes in the original file 
For example 

In [5]:
attributes["OID_"][82427:82430] 


82427            82428
98304     is a dup (1)
98305            82429
Name: OID_, dtype: object

Check for "OID_" values that are 8 charachters or more and likely invalid. 


In [6]:
# Check for weird OID values 
to_drop = []
OID_list = attributes["OID_"].tolist()
   

for i in range (attributes.shape[0]):
    OID = OID_list[i]
    if len(str(OID)) > 7:
        print("at row", i, "OID id:", OID)
        to_drop.append(i)
        
print("")
print ("Row indices to drop:")
to_drop        
        


at row 82428 OID id:  is a dup (1)
at row 103747 OID id:  AL3370708555920130221
at row 103748 OID id:  AL3377808553420130128
at row 103749 OID id:  AL3374308553420130218
at row 103750 OID id:  AL3373008554720130218
at row 103751 OID id:  Are dups (1)
at row 104567 OID id:  small subsequent burned area. (1)
at row 104575 OID id:  GA3374908328320150121
at row 104576 OID id:  dups (1)
at row 104792 OID id:  dup (1)
at row 116877 OID id:  AR3574809291720130327); also adjacent to AR3570009286920130411. (1)

Row indices to drop:


[82428,
 103747,
 103748,
 103749,
 103750,
 103751,
 104567,
 104575,
 104576,
 104792,
 116877]

Delete the rows with nonconforming indices 

In [7]:
 attributes = attributes.drop(attributes.index[to_drop], axis=0)
attributes.shape    


(135061, 6)

Now, clean up some columns with mixed datatypes

In [8]:
# the attributes OID colum is a mix of string and int, cooerce to numeric
attributes['OID_'] = pd.to_numeric(attributes['OID_'], errors='coerce', downcast='integer')

In [9]:
# the Fire Year column is a mix of string and int, cooerce to numeric
attributes['Fire_Year'] = pd.to_numeric(attributes['Fire_Year'], errors='coerce', downcast='integer')

# Step 2 Load and Concat Fire Distance Files 

Load in fire data.  The code is flexible incase future datasets increase in size, but my datapull only had batches 2-27. 

In [10]:
f = "Data/fires_distance_2.csv"
fire_dist = pd.read_csv(f)
fire_dist.head()

,OBJECTID,Distance to Kearny,Closest long,Closest lat
0,13077,1235.364800,-120.925443,35.655980
1,13078,1118.345499,-120.153343,39.383876
2,13079,1146.742793,-120.644932,39.199013
3,13080,2918.436681,-161.011130,65.320212
4,13081,1133.013745,-118.220333,34.433047


In [11]:
for batch_no in range (3, 30):
    try:
        f = f"Data/fires_distance_{batch_no}.csv"
        new = pd.read_csv(f)
        fire_dist = pd.concat([fire_dist, new], axis=0)
    except: 
        print("Error adding batch no", batch_no)
        print("Possible batch doesn't exist")
        
fire_dist.shape

Error adding batch no 28
Possible batch doesn't exist
Error adding batch no 29
Possible batch doesn't exist


(119030, 4)

Looks like we got everything in batches 2-27, as desired. 

# Step 3 merge the attributes dataset and the distances dataset

To merge attributes and distance data, join on the "OBJECTID" feild from the distance data to the "OID_" field in the attributes data.  

In [12]:
# Now merge the attributes dataset and the distances dataset

col1 = "OID_"
col2 = "OBJECTID"

merged_df = attributes.merge(fire_dist, left_on=col1, right_on=col2, how='left')
merged_df[90000:90010]


,OID_,Assigned_Fire_Type,Fire_Year,GIS_Acres,Listed_Fire_Names,Listed_Fire_Dates,OBJECTID,Distance to Kearny,Closest long,Closest lat
90000,90001,Likely Wildfire,2004,133.415596,Ponderosa (3),NaN,90001.0,793.245478,-111.109134,34.307069
90001,90002,Likely Wildfire,2004,126.151102,Cave Springs (1),NaN,90002.0,823.985696,-84.575482,36.923435
90002,90003,Likely Wildfire,2004,116.749996,"WILLIAMS BUTTE (2), Williams Butte (1)",NaN,90003.0,1168.859894,-120.739282,47.427621
90003,90004,Wildfire,2004,191.376117,"STANSBURY ISLAND (3), TAB (3), TABBY (3), Stan...",Listed Wildfire Containment Date(s): 2004-10-1...,90004.0,703.664808,-112.502450,40.778192
90004,90005,Likely Wildfire,2004,114.275132,Finally (1),NaN,90005.0,841.779135,-84.326940,36.734490
90005,90006,Likely Wildfire,2004,112.485449,"SOUTH 1 (2), South 1 (1)",NaN,90006.0,833.861342,-113.877611,37.108638
90006,90007,Likely Wildfire,2004,110.973069,Rail Hollow (1),NaN,90007.0,1055.039007,-79.595843,38.152189
90007,90008,Likely Wildfire,2004,107.175071,Lime Kiln (1),NaN,90008.0,1050.235309,-79.737280,38.014634
90008,90009,Wildfire,2004,105.966790,Crooked Creek (2),Listed Ignition Date(s): 2004-03-25 (2) | List...,90009.0,639.625029,-107.947011,47.423763
90009,90010,Likely Wildfire,2004,102.174906,Ingram Draft (1),NaN,90010.0,1061.071321,-79.517863,38.049173


In [13]:
# Find instances where there was an attribute entry but no distance entry
# print out the OID of these 

#merged_df = merged_df.iloc[90000:100000]

missing_dist  = merged_df[merged_df[col2].isnull()]["OID_"]
print(len(missing_dist))


16031


About 16K values with no distances sounds right.  We dropped the first ~12K because no year, and several other small blocks later in the data pull. 

In [14]:
merged_df.shape

(135061, 10)

In [15]:
# Find instances where there was a distance entry
# but no attribute entry
# print out the OBJECTID of these 

OBJECTID_all = fire_dist["OBJECTID"].drop_duplicates().tolist()
OBJECTID_merged = merged_df["OBJECTID"].drop_duplicates().tolist()
print(len(OBJECTID_all), len(OBJECTID_merged))

#missing_dist = []
#i = 0
#for obj in OBJECTID_all:
    #i+=1
    #if (i%1000 == 0):
        #print("weve checked", i)
    #if not obj in OBJECTID_merged:
        #print("Missing:", obj)
        #missing_dist.append(obj)

#missing_dist        


119030 119031


There are no fire distance datapoints that were not merged.  There is one extra value for OBJECTID in the merged dataset because some entries have "NAN", representing there was no fire dist to merge. 

# Step 4: Polish the Merged Dataset 
In the next code blocks, we drop data that doesn't have an object ID or distance value and fires before 1963.
Finally, we reset the index. 


In [16]:
# drop any rows that don't have an object ID 
final = merged_df[~merged_df["OBJECTID"].isna()]
final.shape

(119030, 10)

In [17]:
# drop any remaining rows that don't have a distance (these will be from "curvedRings")
final = final[~final["Distance to Kearny"].isna()]
final.shape

(118995, 10)

Looks like we had ~35 drops due to "curvedRings".  Sounds about right. 

Now drop fires that were too far away 

In [18]:
# Now we will drop any fires where distance > 1250
too_far = final["Distance to Kearny"] > 1250
print(f"Of the {len(too_far)} fires, there were {sum(too_far)} out of the 1250 mile range")
final = final[~too_far]
final.shape

Of the 118995 fires, there were 16725 out of the 1250 mile range


(102270, 10)

Now drop fires that were earlier than 1963

In [19]:
# Now drop any years less than 1963
too_early = final["Fire_Year"] < 1963
print(f"Of the {len(too_early)} fires left, there were {sum(too_early)} earlier than 1963")
final = final[~too_early]
final.shape


Of the 102270 fires left, there were 1298 earlier than 1963


(100972, 10)

In [20]:
# reset index 
final.set_index('OID_', inplace=True)

# Step 4: Save Cleaned and Merged Data to File 

In [21]:
f= "Data/fire_dat_merged.csv"
final.to_csv(f)